In [1]:
!pip install --upgrade google-cloud-speech

     |████████████████████████████████| 112kB 821kB/s eta 0:00:01
     |████████████████████████████████| 102kB 1.4MB/s ta 0:00:01
     |████████████████████████████████| 512kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.5MB/s eta 0:00:01
     |████████████████████████████████| 143kB 1.4MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 887kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/03/74/3956721ea1eb4bcf7502a311fdaa60b85bd751de4e57d1943afe9b334141/googleapis_common_protos-1.52.0-py2.py3-none-any.whl
     |████████████████████████████████| 3.8MB 845kB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/cb/3f/2f35751408c0b2a425aa48dd79301a48418858d3366f84c75d1aa42b6e2d/PyYAML-5.4.1-cp38-cp38-macosx_10_9_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/95/de/214830a981892a3e286c3794f41ae67a4495df1108c3da8a9f62159b9a9d/pyasn1_modules-0.2.8-py2.py3-none-any.whl
  Using cached https://files.pyth

In [64]:
!pip install --upgrade google-cloud-storage

     |████████████████████████████████| 102kB 1.1MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/22/52/4b086e0d15245b648e2b6d408fb2f2974a3a5a405de5d7fae25cd085a3fa/google_resumable_media-1.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ad/fc/6e8c449185cb8862af353c1164100ff75e32d55ba1de3baf9eaa01b7d2a9/google_cloud_core-1.6.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f7/d8/4b5b7c93b38cf7bed500830bb4b3ba116f6f9b966c14c636f12c56f6cd37/google_crc32c-1.1.2-cp38-cp38-macosx_10_14_x86_64.whl
You should consider upgrading via the 'pip install --upgrade pip' command.


In [52]:
PROJECT_ID = 'your-project-id'  # <---CHANGE THIS
var = "https://console.developers.google.com/apis/api/speech.googleapis.com/overview?project="+PROJECT_ID
print("If this is the first time you are using the API in the project please enable the API via this link:\n"+ var)

If this is the first time you are using the API in the project please enable the API via this link:
https://console.developers.google.com/apis/api/speech.googleapis.com/overview?project=your-project-id


# Make sure to enable the API
<img src="enable-api.png" width="800">

In [29]:
!export GOOGLE_APPLICATION_CREDENTIALS="/AutoMLdemo-64ae368c71e2.json"

In [30]:
GOOGLE_APPLICATION_CREDENTIALS

'AutoMLdemo-64ae368c71e2.json'

In [35]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="AutoMLdemo-64ae368c71e2.json"
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS'])

AutoMLdemo-64ae368c71e2.json


In [66]:
# Imports the Google Cloud client library
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = "test-bucket-speech-n"

# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name)

print("Bucket {} created.".format(bucket.name))

Bucket test-bucket-speech-n created.


In [67]:
from google.cloud import storage


def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # bucket_name = "your-bucket-name"
    # source_file_name = "local/path/to/file"
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )
upload_blob(bucket_name, "Armstrong_Small_Step.ogg.mp3", "Armstrong_Small_Step.ogg.mp3")

File Armstrong_Small_Step.ogg.mp3 uploaded to Armstrong_Small_Step.ogg.mp3.


In [68]:
!gsutil ls gs://test-bucket-speech-n



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

gs://test-bucket-speech-n/Armstrong_Small_Step.ogg.mp3


In [73]:
from google.cloud import speech_v1p1beta1 as speech


def sample_recognize(storage_uri):
    """
    Performs synchronous speech recognition on an audio file

    Args:
      storage_uri URI for audio file in Cloud Storage, e.g. gs://[BUCKET]/[FILE]
    """

    client = speech.SpeechClient()

    storage_uri = 'gs://test-bucket-speech-n/Armstrong_Small_Step.ogg.mp3'

    # The language of the supplied audio
    language_code = "en-US"

    # Sample rate in Hertz of the audio data sent
    sample_rate_hertz = 44100

    # Encoding of audio data sent. This sample sets this explicitly.
    # This field is optional for FLAC and WAV audio formats.
    encoding = speech.RecognitionConfig.AudioEncoding.MP3
    config = {
        "language_code": language_code,
        "sample_rate_hertz": sample_rate_hertz,
        "encoding": encoding,
        "model": "video"
    }
    audio = {"uri": storage_uri}

    response = client.recognize(config=config, audio=audio)
    print(response)
    for result in response.results:
        # First alternative is the most probable result
        alternative = result.alternatives[0]
        print(u"Transcript: {}".format(alternative.transcript))
        
sample_recognize("Armstrong_Small_Step.ogg.mp3")

results {
  alternatives {
    transcript: "step off the Lem now"
    confidence: 0.8174150586128235
  }
  language_code: "en-us"
}
results {
  alternatives {
    transcript: " that\'s one small step for man"
    confidence: 0.8332863450050354
  }
  language_code: "en-us"
}
results {
  alternatives {
    transcript: " one giant leap for mankind"
    confidence: 0.9128385782241821
  }
  language_code: "en-us"
}

Transcript: step off the Lem now
Transcript:  that's one small step for man
Transcript:  one giant leap for mankind
